In [59]:
!pip install requests pandas numpy selenium imdbpy webdriver-manager

In [40]:
import requests
from bs4 import BeautifulSoup

def get_imdb_reviews(movie_id):
    base_url = f"https://www.imdb.com/title/{movie_id}/reviews"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    reviews = []
    pagination_key = ''

    while True:
        url = base_url if not pagination_key else f"{base_url}/_ajax?paginationKey={pagination_key}"
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to retrieve page. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.content, 'html.parser')
        review_elements = soup.find_all('article', class_='sc-7d2e5b85-1 cvfQlw user-review-item')

        for review in review_elements:
            title_tag = review.find('div', class_='ipc-title')
            content_tag = review.find('div', class_='ipc-html-content-inner-div')
            rating_tag = review.find('span', class_='ipc-rating-star--rating')
            helpful_count_tag = review.find('span', class_='ipc-voting__label__count ipc-voting__label__count--up')
            not_helpful_count_tag = review.find('span', class_='ipc-voting__label__count ipc-voting__label__count--down')

            title = title_tag.get_text(strip=True) if title_tag else 'No title'
            content = content_tag.get_text(strip=True) if content_tag else 'No content'
            rating = rating_tag.get_text(strip=True) if rating_tag else 'No rating'
            helpful_count = helpful_count_tag.get_text(strip=True) if helpful_count_tag else '0'
            not_helpful_count = not_helpful_count_tag.get_text(strip=True) if not_helpful_count_tag else '0'

            reviews.append({
                'title': title,
                'content': content,
                'rating': rating,
                'helpful_count': helpful_count,
                'not_helpful_count': not_helpful_count
            })

        load_more_data = soup.find('div', class_='load-more-data')
        if not load_more_data:
            break
        pagination_key = load_more_data['data-key']

    return reviews

# Przykład użycia
movie_id = 'tt0111161'  # ID filmu "The Shawshank Redemption"
reviews = get_imdb_reviews(movie_id)
for i, review in enumerate(reviews, 1):
    print(f"Review {i}:")
    print(f"Title: {review['title']}")
    print(f"Content: {review['content']}")
    print(f"Rating: {review['rating']}")
    print(f"Helpful Count: {review['helpful_count']}")
    print(f"Not Helpful Count: {review['not_helpful_count']}\n")

Review 1:
Title: Some birds aren't meant to be caged.
Content: No content
Rating: 10
Helpful Count: 0
Not Helpful Count: 0

Review 2:
Title: An incredible movie. One that lives with you.
Content: It is no wonder that the film has such a high rating, it is quite literally breathtaking. What can I say that hasn't said before? Not much, it's the story, the acting, the premise, but most of all, this movie is about how it makes you feel. Sometimes you watch a film, and can't remember it days later, this film loves with you, once you've seen it, you don't forget.The ultimate story of friendship, of hope, and of life, and overcoming adversity.I understand why so many class this as the best film of all time, it isn't mine, but I get it. If you haven't seen it, or haven't seen it for some time, you need to watch it, it's amazing. 10/10.
Rating: 10
Helpful Count: 0
Not Helpful Count: 0

Review 3:
Title: Don't Rent Shawshank.
Content: I'm trying to save you money; this is the last film title that

In [81]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

def get_imdb_reviews(movie_id):
    base_url = f"https://www.imdb.com/title/{movie_id}/reviews"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    reviews = []

    # Use Selenium to click the "All" button
    PATH = r"C:\chromedriver-win64\chromedriver.exe"
    service = Service(PATH)
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run in headless mode
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(base_url)
    driver.implicitly_wait(2)  # Wait for the page to load

    # Click the "All" button to load all reviews
    try:
        load_more_button = driver.find_element(By.CSS_SELECTOR, "button.ipc-btn.ipc-btn--single-padding.ipc-btn--center-align-content.ipc-btn--default-height.ipc-btn--core-base.ipc-btn--theme-base.ipc-btn--button-radius.ipc-btn--on-accent2.ipc-text-button.ipc-see-more__button")
        load_more_button.click()
        time.sleep(2)  # Wait for the reviews to load
    except Exception as e:
        print(f"Error clicking 'All' button: {e}")

    # Get the page content after clicking the "All" button
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    review_elements = soup.find_all('article', class_='sc-7d2e5b85-1 cvfQlw user-review-item')

    for review in review_elements:
        title_tag = review.find('div', class_='ipc-title')
        content_tag = review.find('div', class_='ipc-html-content-inner-div')
        rating_tag = review.find('span', class_='ipc-rating-star--rating')
        helpful_count_tag = review.find('span', class_='ipc-voting__label__count ipc-voting__label__count--up')
        not_helpful_count_tag = review.find('span', class_='ipc-voting__label__count ipc-voting__label__count--down')

        title = title_tag.get_text(strip=True) if title_tag else 'No title'
        content = content_tag.get_text(strip=True) if content_tag else 'No content'
        rating = rating_tag.get_text(strip=True) if rating_tag else 'No rating'
        helpful_count = helpful_count_tag.get_text(strip=True) if helpful_count_tag else '0'
        not_helpful_count = not_helpful_count_tag.get_text(strip=True) if not_helpful_count_tag else '0'

        reviews.append({
            'title': title,
            'content': content,
            'rating': rating,
            'helpful_count': helpful_count,
            'not_helpful_count': not_helpful_count
        })

    return reviews

# Example usage
movie_id = 'tt0111161'  # ID for "The Shawshank Redemption"
reviews = get_imdb_reviews(movie_id)
for i, review in enumerate(reviews, 1):
    print(f"Review {i}:")
    print(f"Title: {review['title']}")
    print(f"Content: {review['content']}")
    print(f"Rating: {review['rating']}")
    print(f"Helpful Count: {review['helpful_count']}")
    print(f"Not Helpful Count: {review['not_helpful_count']}\n")

Error clicking 'All' button: Message: no such element: Unable to locate element: {"method":"css selector","selector":"button.ipc-btn.ipc-btn--single-padding.ipc-btn--center-align-content.ipc-btn--default-height.ipc-btn--core-base.ipc-btn--theme-base.ipc-btn--button-radius.ipc-btn--on-accent2.ipc-text-button.ipc-see-more__button"}
  (Session info: chrome=133.0.6943.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7233FC6A5+28789]
	(No symbol) [0x00007FF723365B20]
	(No symbol) [0x00007FF7231F8F9A]
	(No symbol) [0x00007FF72324F346]
	(No symbol) [0x00007FF72324F57C]
	(No symbol) [0x00007FF7232A2B17]
	(No symbol) [0x00007FF72327736F]
	(No symbol) [0x00007FF72329F7E3]
	(No symbol) [0x00007FF723277103]
	(No symbol) [0x00007FF72323FFC0]
	(No symbol) [0x00007FF723241273]
	GetHandleVerifier [0x00007FF723741AED+3458237]
	GetHandleVerifier [0x00007FF7237582